In [1]:
import tkinter as tk
from tkinter import filedialog
import socket
import threading
import cv2
import os
from stegano import lsb

def send_message():
    mode = mode_var.get()
    message = message_entry.get()

    if mode == "image":
        file_types = [("Image Files", "*.png *.jpg *.jpeg")]
    elif mode == "video":
        file_types = [("Video Files", "*.mp4 *.avi *.mov")]
    else:
        file_types = [("Audio Files", "*.mp3 *.wav *.ogg")]

    file_path = filedialog.askopenfilename(filetypes=file_types)

    if file_path:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_address = ('localhost', 8000)
        client_socket.connect(server_address)

        client_socket.sendall(mode.encode())

        if mode == "image":
            encoded_image = lsb.hide(file_path, message, auto_convert_rgb=True)
            encoded_image.save("encoded_img.png")
            with open("encoded_img.png", 'rb') as file:
                file_read = file.read()
        elif mode == "video":
            cap = cv2.VideoCapture(file_path)
            counter = 0
            frames = []
            client_path = "./client-2/video"
            #if video folder not present, create it
            if not os.path.exists(client_path):
                os.mkdir(client_path)
            #if present remove all files
            else:
                for file in os.listdir(client_path):
                    os.remove(os.path.join(client_path, file))

            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                #create images from frames
                cv2.imwrite(f"{client_path}/frame_{counter}.png", frame)
                counter += 1

            encoded_frame = lsb.hide(f"{client_path}/frame_0.png", message, auto_convert_rgb=True)
            #save the encoded frame
            encoded_frame.save(f"{client_path}/frame_0.png")
            #create video using images
            for i in range(counter):
                img = cv2.imread(f"{client_path}/frame_{i}.png")
                height, width, layers = img.shape
                if i == 0:
                    size = (width,height)
                frames.append(img)

            out = cv2.VideoWriter(f'{client_path}/output.avi',cv2.VideoWriter_fourcc(*'FFV1'), cap.get(cv2.CAP_PROP_FPS), size)
            for i in range(len(frames)):
                out.write(frames[i])
            out.release()
            #read the video
            with open(f'{client_path}/output.avi', "rb") as video:
                file_read = video.read()


        client_socket.sendall(file_read)
        client_socket.close()
    else:
        response_label.configure(text="No file selected.")
    return

def receive_thread():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 8001)
    server_socket.bind(server_address)
    server_socket.listen(1)
    message = ""
    
    while True:
        client_socket, client_address = server_socket.accept()

        mode = client_socket.recv(1024).decode()

        if mode == "image":
            file = open("received_img.png", "wb")
            data = client_socket.recv(1024)
            while data:
                file.write(data)
                data = client_socket.recv(1024)
            file.close()

            encoded_image = "received_img.png"
            decoded_message = lsb.reveal(encoded_image)
            message = decoded_message
        elif mode == "video":
            client_path = "./client-2/video"
            #if video folder not present, create it
            if not os.path.exists(client_path):
                os.mkdir(client_path)
            #if present remove all files
            else:
                for file in os.listdir(client_path):
                    os.remove(os.path.join(client_path, file))
                    
            file = open(f"{client_path}/received_video.avi", "wb")
            data = client_socket.recv(1024)
            while data:
                file.write(data)
                data = client_socket.recv(1024)
            file.close()
            
            cap = cv2.VideoCapture(f"{client_path}/received_video.avi")
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(f"{client_path}/frame_0.png", frame)
                #decode the message
                decoded_message = lsb.reveal(f"{client_path}/frame_0.png")
                message = decoded_message
                cap.release()
                    
        received_label.configure(text=f"Received message: {message}")
        mode_label.configure(text=f"Mode of encoding: {mode}")

    client_socket.close()

# Creating the client UI
client = tk.Tk()
client.title("Client - 2")
client.geometry("400x200")  # Set the width to 800 and height to 600

# Message input
message_label = tk.Label(client, text="Enter your message:")
message_label.pack()
message_entry = tk.Entry(client)
message_entry.pack()

# Message input
message_label = tk.Label(client, text="Select a file:")
message_label.pack()

# Mode of encoding input
mode_label = tk.Label(client, text="Select the mode of encoding:")
mode_label.pack()

# Mode of encoding radio buttons
mode_var = tk.StringVar(value="text")
image_radio = tk.Radiobutton(client, text="Image", variable=mode_var, value="image")
image_radio.pack()  
video_radio = tk.Radiobutton(client, text="Video", variable=mode_var, value="video")
video_radio.pack()
text_radio = tk.Radiobutton(client, text="Audio", variable=mode_var, value="audio")
text_radio.pack()

# Send button
send_button = tk.Button(client, text="Send", command=send_message)
send_button.pack()

# Received message label
received_label = tk.Label(client, text="Received message:")
received_label.pack()

# Response label
response_label = tk.Label(client, text="")
response_label.pack()

# Start the receive thread
receive_thread = threading.Thread(target=receive_thread)
receive_thread.daemon = True
receive_thread.start()

# Start the client UI
client.mainloop()
